Using **Pipeline** from scikit-learn allows you to streamline the process of building<br>and evaluating machine learning models by chaining together preprocessing steps and model training into a single object.<br> This helps ensure that the transformations are applied consistently and in the correct order.

**Key Features of Pipeline**<br>
_Sequential Processing:_ Each step in a pipeline is executed in sequence, making it easy to manage the flow of data from preprocessing to model fitting.

_Automatic Parameter Tuning:_<br> When using tools like GridSearchCV, the pipeline ensures that preprocessing steps are applied during cross-validation, making hyperparameter tuning more straightforward.

_Simplified Code:_ It reduces boilerplate code by encapsulating preprocessing and model fitting in one cohesive structure.

_Easier Model Management:_ You can save and load pipelines easily, facilitating model deployment and reproducibility.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ucimlrepo import fetch_ucirepo 

from sklearn.metrics import r2_score 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

In [13]:
# fetch dataset 
concrete_compressive_strength = fetch_ucirepo(id=165)

In [14]:

# data (as pandas dataframes) 
X = concrete_compressive_strength.data.features 
y = concrete_compressive_strength.data.targets 

type(X), type(y)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [15]:
# Metadata and variable info 
#print(concrete_compressive_strength.metadata)  
#print(concrete_compressive_strength.variables)

In [16]:
entire_data = pd.concat([X,y], axis=1)
entire_data

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, y_train.shape

((824, 8), (824, 1))

In [7]:
#The make_column_transformer:
#function in scikit-learn is a utility that helps you create a composite transformer
#for preprocessing data in machine learning pipelines.

#make_column_selector: The function returns a callable (a function)
#that can be used to select columns from a DataFrame.

ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

ct = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)), #'drop': Drops those columns.
                                                                #'passthrough': Leaves those columns unchanged.
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False,
).set_output(transform='pandas')


In [19]:
ss = StandardScaler()

mm = MinMaxScaler()

pf = PolynomialFeatures(degree=3, include_bias=False).set_output(transform='pandas')

lr = LinearRegression()

In [22]:
pipe = Pipeline([
    ('column_transform',ct), 
    ('standard_scaler',ss),
    # ('min_max_scaler',mm),
    ('polynomial_features',pf),
    ('linear_regression',lr)
])



In [21]:
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

r2_score(y_test, y_pred)

0.8058981526082563